# Часть 1. Проверка гипотезы в Python и составление аналитической записки

# Сравнение активности пользователей Яндекс Книг в Москве и Санкт-Петербурге

- Автор: Рунушкина Ольга Андреевна 
- Дата: 14.04.2025

## Цели и задачи проекта

**Цель проекта**:
Проверить гипотезу о том, что пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении Яндекс Книги, чем пользователи из Москвы.

**Задачи проекта**:

- Загрузить и предобработать данные (в том числе проверить наличие дубликатов).
- Сравнить размеры групп и описательные статистики.
- Провести статистическую проверку гипотезы с помощью t-теста.
- Сформулировать выводы на основе результатов тестирования.

## Описание данных

В датасете представлены данные пользователей из двух городов — Москвы и Санкт-Петербурга.
Каждая строка содержит информацию:

`city` - город пользователя;

`puid` - уникальый идентификаторй;

`hours` - сумма времени, потраченного на чтение и прослушивание книг.

## Содержимое проекта

1. Импорт библиотек и загрузка CSV-файла с данными.
2. Проверка и удаление дубликатов по puid.
3. Сравнение размеров выборок и описательных статистик (среднее, медиана, стандартное отклонение и др.).
4. Проведение одностороннего t-теста для независимых выборок:
- Нулевая гипотеза: средняя активность пользователей в СПб не больше, чем в Москве.
- Альтернативная гипотеза: средняя активность в СПб больше, чем в Москве.
5. Получение p-value и интерпретация результата.
6. Аналитическая записка с выводами и возможными причинами.

## 1. Загрузка данных и знакомство с ними

Загрузите данные пользователей из Москвы и Санкт-Петербурга c их активностью (суммой часов чтения и прослушивания) из файла `/datasets/yandex_knigi_data.csv`.

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.proportion import proportions_ztest

In [2]:
# Выгружаем данные
df = pd.read_csv('/datasets/yandex_knigi_data.csv')

In [3]:
# Выводим первые 5 строк
df.head()

,Unnamed: 0,city,puid,hours
0,0,Москва,9668,26.167776
1,1,Москва,16598,82.111217
2,2,Москва,80401,4.656906
3,3,Москва,140205,1.840556
4,4,Москва,248755,151.326434


In [4]:
# Выводим информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8784 non-null   int64  
 1   city        8784 non-null   object 
 2   puid        8784 non-null   int64  
 3   hours       8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


Датафрейм имеет 4 столбца и 8560 строк 
Пропусков не найдено, типы данных корректны

Теперь проверим столбец с индификаторами пользователей на наличие дубликатов, чтобы избежать искажение результатов

In [5]:
# Проверяем столбец на дубликаты
df['puid'].duplicated().sum()

244

In [6]:
# Удаляем дубликаты
df.drop_duplicates(subset='puid', inplace=True)

In [7]:
# Проверяем результат
df['puid'].duplicated().sum()

0

## 2. Проверка гипотезы в Python

Гипотеза звучит так: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы. Попробуйте статистически это доказать, используя одностороннюю проверку гипотезы с двумя выборками:

- Нулевая гипотеза H₀: Средняя активность пользователей в часах в Санкт-Петербурге не больше, чем в Москве, имеющееся различие статистически незначимо.

- Альтернативная гипотеза H₁: Средняя активность пользователей в Санкт-Петербурге больше, и это различие статистически значимо.

In [8]:
# Смотрим размеры выборок
df['city'].value_counts()

Москва             6234
Санкт-Петербург    2306
Name: city, dtype: int64

In [9]:
# Выводим статистические покаатели
df.groupby('city')['hours'].describe()

,count,mean,std,min,25%,50%,75%,max
city,,,,,,,,
Москва,6234.0,10.881092,36.851683,0.000018,0.059903,0.924498,5.939972,857.209373
Санкт-Петербург,2306.0,11.264433,39.831755,0.000025,0.060173,0.875355,6.138424,978.764775


На основе анализа можно сделать следующие выводы:
- Данные и в Москве и в Санкт-Петербурге распределены ассиметрично
- Медиана сильно меньше среднего значения, что говорит о выбросах. Это также подтверждает огромные максимальные показатели

Значит для проверки гипотез нужно использовать тест, не чувствительный к выбросам и не требущий нормального распределения. **С этим справится непараметрический тест**

In [10]:
# Формируем группы
moscow = df[df['city'] == 'Москва']['hours']
spb = df[df['city'] == 'Санкт-Петербург']['hours']

alpha = 0.05

# Проводим статистический тест
stat, p_value = mannwhitneyu(spb, moscow, alternative='greater')
print(f'p_value = {p_value}')

if p_value < alpha:
    print('Разница статистически значимая. Альтернативная гипотеза подтверждается')
else:
    print('Разница статистически незначительна. Нулевая гипотеза подтверждается')

p_value = 0.6189600878052618
Разница статистически незначительна. Нулевая гипотеза подтверждается


## 3. Аналитическая записка
По результатам анализа данных подготовьте аналитическую записку, в которой опишете:

- Выбранный тип t-теста и уровень статистической значимости.

- Результат теста, или p-value.

- Вывод на основе полученного p-value, то есть интерпретацию результатов.

- Одну или две возможные причины, объясняющие полученные результаты.



Для проверки гипотезы о различии средней активности пользователей в приложении между Москвой и Санкт-Петербургом был проведён **тест МУ**. Уровень статистической значимости был установлен на уровне **а = 0.05**.

В результате теста было получено значение **p-value = 0.3382**, что значительно превышает пороговое значение **a**. Это означает, что нет оснований отвергнуть нулевую гипотезу. Следовательно, статистически значимых различий в средней активности пользователей между Москвой и Санкт-Петербургом **не выявлено**.

Возможные причины полученного результата могут быть следующими:

- Пользователи из двух городов действительно проводят схожее количество времени в приложении, и различия носят случайный характер.
- Объём выборки или вариативность данных может быть недостаточной для выявления более тонких различий в поведении пользователей.
- Аудитории в Москве и Санкт-Петербурге могут иметь схожий стиль использования приложения (например, одинаковые часы активности, цели использования и пр.).
- Контент и функциональность приложения могут одинаково хорошо подходить обеим аудиториям, нивелируя возможные различия.


**Таким образом, гипотеза о том, что пользователи из Санкт-Петербурга в среднем более активны, не подтверждается статистически**.

# Часть 2. Анализ результатов A/B-тестирования

Теперь вам нужно проанализировать другие данные. Представьте, что к вам обратились представители интернет-магазина BitMotion Kit, в котором продаются геймифицированные товары для тех, кто ведёт здоровый образ жизни. У него есть своя целевая аудитория, даже появились хиты продаж: эспандер со счётчиком и напоминанием, так и подстольный велотренажёр с Bluetooth.

В будущем компания хочет расширить ассортимент товаров. Но перед этим нужно решить одну проблему. Интерфейс онлайн-магазина слишком сложен для пользователей — об этом говорят отзывы.

Чтобы привлечь новых клиентов и увеличить число продаж, владельцы магазина разработали новую версию сайта и протестировали его на части пользователей. По задумке, это решение доказуемо повысит количество пользователей, которые совершат покупку.

Ваша задача — провести оценку результатов A/B-теста. В вашем распоряжении:

* данные о действиях пользователей и распределении их на группы,

* техническое задание.

Оцените корректность проведения теста и проанализируйте его результаты.

## 1. Опишите цели исследования.



**Цель исследования**: оценка влияния нового пользовательского интерфейса интернет-магазина BitMotion Kit на поведение пользователей. В частности, проверяется гипотеза о том, что обновлённый, более простой интерфейс способствует увеличению конверсии зарегистрированных пользователей в покупателей в течение первых 7 дней после регистрации.

Для проверки этой гипотезы была проведена A/B-тестовая кампания, в рамках которой пользователи были случайным образом разделены на две группы:

- Группа A (контрольная) — использовала текущую версию интерфейса.
- Группа B (тестовая) — взаимодействовала с новой версией интерфейса.

Главной метрикой исследования является **конверсия в покупку** в течение недели после регистрации.

## 2. Загрузите данные, оцените их целостность.


In [11]:
# Выгружаем данные
participants = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_participants.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_events.zip',
                     parse_dates=['event_dt'], low_memory=False)

In [12]:
# Выгружаем первые строки 
participants.head()

,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


In [13]:
# Смотрим информацию о датафрейме
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


Датафрейм имеет 4 столбца и 14525 строк Пропусков не найдено, типы данных корректны

In [14]:
# Проверяем столбец с пользователями на дубликаты
participants['user_id'].duplicated().sum()

887

In [15]:
# Выводим первые строки
events.head()

,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN


In [16]:
# Смотрим основную информацию о датафрейме
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


In [17]:
# Проверяем на дубликаты
events.duplicated().sum()

36318

In [18]:
# Удаляем дубликаты
events.drop_duplicates(inplace=True)

In [19]:
# Проверяем результат
events.duplicated().sum()

0

В датафрейме 4 столбца и 787286 строк. В столбце `details`  обнаружено 538262 пропуска. Скорее всего это связано с тем, что просто нет дополнительных данных. В рамках проекта этот столбец для анализа не понадобится, поэтому оставим как есть.

## 3. По таблице `ab_test_participants` оцените корректность проведения теста:

   3\.1 Выделите пользователей, участвующих в тесте, и проверьте:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [20]:
# Создаем отфильтрованный по тесту датафрейм
filtered_participants = participants[participants['ab_test'] == 'interface_eu_test']

In [21]:
# Смотрим равномерность распределения пользователей по группам
filtered_participants['group'].value_counts(normalize=True)

B    0.503871
A    0.496129
Name: group, dtype: float64

Пользователи равномерно распределены по группам.

In [22]:
# Формируем группы
users_a = filtered_participants[filtered_participants['group'] == 'A']['user_id']
users_b = filtered_participants[filtered_participants['group'] == 'B']['user_id']

# Ищем пересечения
intersection = list(set(users_a) & set(users_b))
intersection

[]

In [23]:
#Выделаем пользователей из группы В
users_b = participants[participants['group'] == 'B']

# Группируем по user_id и считаем количество уникальных тестов
multi_test_users = users_b.groupby('user_id')['ab_test'].nunique()

# Оставляем только тех, кто участвовал в более чем одном тесте
multi_test_users = multi_test_users[multi_test_users > 1]

# Посмотрим результат
len(multi_test_users)

116

In [24]:
# Получим список user_id таких пользователей
users_to_remove = multi_test_users.index

# Удалим их из filtered_participants
filtered_participants = filtered_participants[~filtered_participants['user_id'].isin(users_to_remove)]

3\.2 Проанализируйте данные о пользовательской активности по таблице `ab_test_events`:

- оставьте только события, связанные с участвующими в изучаемом тесте пользователями;

In [25]:
# Создаем отфильтрованный датафрейм
filtered_events = events[events['user_id'].isin(filtered_participants['user_id'])]

- определите горизонт анализа: рассчитайте время (лайфтайм) совершения события пользователем после регистрации и оставьте только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [26]:
# Ищем только события с регистрацией
registrations = filtered_events[filtered_events['event_name'] == 'registration']

# Находим для каждого пользователя дату регистарции 
first_reg = registrations.groupby('user_id')['event_dt'].min().reset_index()
first_reg.rename(columns={'event_dt': 'registration_dt'}, inplace=True)

events_merged = filtered_events.merge(first_reg, on='user_id', how='left')

In [27]:
# Вычисляем лайфтайм (в днях)
events_merged['lifetime_days'] = (events_merged['event_dt'] - events_merged['registration_dt']).dt.days

# Оставляем события, совершённые в течение первых 7 дней
events_in_7_days = events_merged[events_merged['lifetime_days'] <= 7]

Оцените достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные параметры:

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.

In [28]:
# Параметры
baseline = 0.3        
mde = 0.03            
alpha = 0.05          
power = 0.8           

# Размер эффекта для бинарной метрики
effect = proportion_effectsize(baseline, baseline + mde)

# Инициализируем расчет мощности
analysis = NormalIndPower()

# Расчет нужного числа пользователей в каждой группе
required_n = analysis.solve_power(effect_size=effect, power=power, alpha=alpha, alternative='two-sided')

print(f"Необходимое количество пользователей в каждой группе: {round(required_n)}")

Необходимое количество пользователей в каждой группе: 3762


- рассчитайте для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [29]:
# Получим список покупателей — кто совершил событие "purchase"
buyers = events_in_7_days[events_in_7_days['event_name'] == 'purchase']['user_id'].unique()

# Добавим колонку "покупал ли" в таблицу участников
filtered_participants['made_purchase'] = filtered_participants['user_id'].isin(buyers)

# Сгруппируем по группам и посчитаем нужное
conversion_summary = filtered_participants.groupby('group').agg(
    total_users=('user_id', 'nunique'),
    buyers=('made_purchase', 'sum')
)

# Добавим столбец с конверсией
conversion_summary['conversion'] = round(conversion_summary['buyers'] / conversion_summary['total_users'], 2)

conversion_summary

,total_users,buyers,conversion
group,,,
A,5383,1542,0.29
B,5351,1665,0.31


- сделайте предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.

В результате A/B-тестирования, направленного на проверку влияния нового интерфейса интернет-магазина, были зафиксированы следующие показатели пользовательской активности:

- В контрольной группе (А):
из 5174 пользователей покупку совершили 1487, что составляет 29%.
- В тестовой группе (B):
из 5229 пользователей покупку совершили 1618, что составляет 31%.

Таким образом, в тестовой группе наблюдается увеличение конверсии на 2 процентных пункта по сравнению с контрольной. Также стоит отметить, что фактический прирост конверсии в 2 п.п. не соответствует запланированному приросту в 3 п.п., поэтому эффект от изменений есть, но недостаточно высокий.

## 4. Проведите оценку результатов A/B-тестирования:

- Проверьте изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез.

**Нулевая гипотеза** 
Конверсия в тестовой группе не увеличилась на 3 процентных пункта по сравнению с контрольной группой.

**Альтернативная гипотеза** 
Конверсия в тестовой группе увеличилась как минимум на 3 процентных пункта по сравнению с контрольной группой.

In [30]:
# Упорядочим таблицу по нужному нам порядку
conversion_ordered = conversion_summary.loc[['B', 'A']]  # важно: B идёт первой

# Получим значения для теста
count = conversion_ordered['buyers'].values
nobs = conversion_ordered['total_users'].values

In [31]:
# Односторонний тест: проверим, стала ли конверсия в группе B значительно выше
z_stat, p_value = proportions_ztest(count, nobs, value=0.03, alternative='larger')

alpha = 0.05

print(f'Z-статистика: {z_stat:.4f}')
print(f'p-value: {p_value:.4f}')

if p_value < alpha:
    print('Изменения статистически значимы. Альтернативная гипотеза подтверждается.')
else:
    print('Изменения статистически незначимы. Нулевая гипотеза пдтверждается.')

Z-статистика: -0.5999
p-value: 0.7257
Изменения статистически незначимы. Нулевая гипотеза пдтверждается.


- Опишите выводы по проведённой оценке результатов A/B-тестирования. Что можно сказать про результаты A/B-тестирования? Был ли достигнут ожидаемый эффект в изменении конверсии?

В рамках проведённого исследования была проверена гипотеза о том, что упрощение интерфейса интернет-магазина BitMotion Kit приведёт к увеличению конверсии пользователей в покупку в течение первых семи дней после регистрации. Для этого был реализован A/B-тест, в котором пользователи случайным образом были распределены на две группы: 
- контрольную (A), использующую текущую версию интерфейса
- и тестовую (B), взаимодействующую с обновлённым вариантом 

По результатам анализа пользовательской активности было установлено, что в контрольной группе из **5174** пользователей покупку совершили **1487**, что составляет **29%** конверсии, тогда как в тестовой группе из **5229** пользователей покупку совершили **1618**, что составляет **31%**. Таким образом, в тестовой группе наблюдается увеличение конверсии на 2 процентных пункта по сравнению с контрольной, что может свидетельствовать о потенциально положительном влиянии нового интерфейса на поведение пользователей. Однако последующая статистическая проверка с использованием Z-теста показала, что наблюдаемая разница не является статистически значимой при проверке прироста именно на 3 процентных пункта (**p-value = 0.8128**). Это означает, что полученные изменения могли возникнуть случайно и не подтверждают устойчивого эффекта от обновления интерфейса.

В связи с этим можно сделать вывод, что на текущем этапе гипотеза не подтверждена. 

Рекомендуется либо повторить эксперимент на более крупной выборке, либо провести дополнительный анализ — например, по сегментам пользователей или типам устройств — чтобы убедиться в наличии или отсутствии эффекта от изменений.

<div style="border:solid Chocolate 2px; padding: 40px">

# Комментарий ревьюера: общий вывод по проекту.

Ольга, проект получился на довольно хорошем уровне - отличная работа над проектом, молодец!

Мне нравится твой аналитический подход к выполнению проекта, ты соблюдаешь структуру работы, выполняешь её последовательно - это очень хорошо! Шаги проекта выполнены по порядку согласно плану проекта, нет смысловых и структурных ям. Важно, что не забываешь про выводы.
    
Над проектом ещё стоит поработать - есть рекомендации по дополнению некоторых твоих шагов проекта. Такие рекомендации я отметил жёлтыми комментариями. Будет здорово, если ты учтёшь их - так проект станет структурно и содержательно более совершенным.
    
Также в работе есть критические замечания. К этим замечаниям я оставил пояснительные комментарии красного цвета, в которых перечислил возможные варианты дальнейших действий. Уверен, ты быстро с этим управишься:)
    
Если о том, что нужно сделать в рамках комментариев, будут возникать вопросы - оставь их, пожалуйста, в комментариях, и я отвечу на них во время следующего ревью.
    
Также буду рад ответить на любые твои вопросы по проекту или на какие-либо другие, если они у тебя имеются - оставь их в комментариях, и я постараюсь ответить:)
    
Жду твой проект на повторном ревью. До встречи:)

<div style="border:solid Chocolate 2px; padding: 40px">

# Комментарий ревьюера: итоговый вывод по проекту.

Ольга, все замечания учтены - проект принят!
    
Спасибо за хорошую работу над проектом, желаю успехов в дальнейшем обучении:)